# Over-fitting and Under-fitting
Underfitting is when a model can still be improved, becuase it's not powerful enough for the situtation, it's over-regularized (over simplified), or maybe it's just not been trained enough. Overfitting on the other hand is when the model starts to memorize the training data and draw patterns that don't (or aren't supposed to) apply to the test data. You have to strike a balance.

In [1]:
import tensorflow as tf
print(tf.__version__)
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import pathlib
import shutil
import tempfile

2.8.0


In [2]:
logdir = pathlib.Path(tempfile.mkdtemp())/"tensorboard_logs"
shutil.rmtree(logdir, ignore_errors=True)

### Higgs dataset
Just a dataset. Using tf's build in function to grab this dataset for us. It contains 11,000,000 examples, each with 28 features, and a binary class label.

In [3]:
gz = tf.keras.utils.get_file('HIGGS.csv.gz', 'http://mlphysics.ics.uci.edu/data/higgs/HIGGS.csv.gz')
FEATURES = 28

This Tf function allows the dataset to be read without first having to be decompressed

In [4]:
ds = tf.data.experimental.CsvDataset(gz,[float()]*(FEATURES+1), compression_type="GZIP")

2022-05-13 13:16:19.770484: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
